In [ ]:
%%sql
-- Ensure schema exist
CREATE SCHEMA IF NOT EXISTS silver;

In [ ]:
%%sql
CREATE MATERIALIZED LAKE VIEW IF NOT EXISTS silver.mlv_customers AS
WITH ca_ranked AS (
  SELECT
    ca.CustomerID,
    ca.AddressID,
    a.AddressLine1,
    a.AddressLine2,
    a.City,
    a.StateProvince,
    a.CountryRegion,
    a.PostalCode,
    ROW_NUMBER() OVER (
      PARTITION BY ca.CustomerID
      ORDER BY greatest(ca.ModifiedDate, a.ModifiedDate) DESC, ca.AddressID DESC
    )                           AS rn,
    greatest(ca.ModifiedDate, a.ModifiedDate) AS ca_addr_modified
  FROM bronze.CustomerAddress ca
  JOIN bronze.Address a
    ON a.AddressID = ca.AddressID
)
SELECT
  c.CustomerID,
  c.Title,
  c.FirstName,
  c.MiddleName,
  c.LastName,
  concat_ws(' ', c.FirstName, c.LastName) AS full_name,
  c.CompanyName,
  lower(c.EmailAddress)                   AS EmailAddress,
  c.Phone,
  r.AddressLine1,
  r.AddressLine2,
  r.City,
  r.StateProvince,
  r.CountryRegion,
  r.PostalCode,
  /* unified watermark across customer + address */
  coalesce(
    greatest(c.ModifiedDate, r.ca_addr_modified),
    c.ModifiedDate,
    r.ca_addr_modified,
    to_timestamp('1900-01-01 00:00:00')
  ) AS row_modified_at
FROM bronze.Customer c
LEFT JOIN ca_ranked r
  ON r.CustomerID = c.CustomerID
 AND r.rn = 1;

In [ ]:
%%sql
CREATE MATERIALIZED LAKE VIEW IF NOT EXISTS silver.mlv_products AS
SELECT
  p.ProductID,
  p.ProductNumber,
  trim(p.Name)                      AS ProductName,
  p.Color,
  p.Size,
  p.StandardCost,
  p.ListPrice,
  pc.ProductCategoryID,
  trim(pc.Name)                     AS ProductCategory,
  pm.ProductModelID,
  trim(pm.Name)                     AS ProductModel,
  /* watermark */
  coalesce(
    greatest(p.ModifiedDate, pc.ModifiedDate, pm.ModifiedDate),
    p.ModifiedDate, pc.ModifiedDate, pm.ModifiedDate,
    to_timestamp('1900-01-01 00:00:00')
  ) AS row_modified_at
FROM bronze.Product p
LEFT JOIN bronze.ProductCategory pc
  ON pc.ProductCategoryID = p.ProductCategoryID
LEFT JOIN bronze.ProductModel pm
  ON pm.ProductModelID = p.ProductModelID;

In [ ]:
%%sql
CREATE MATERIALIZED LAKE VIEW IF NOT EXISTS silver.mlv_addresses AS
SELECT
  a.AddressID,
  a.AddressLine1,
  a.AddressLine2,
  a.City,
  a.StateProvince,
  a.CountryRegion,
  a.PostalCode,
  COALESCE(a.ModifiedDate, to_timestamp('1900-01-01 00:00:00')) AS row_modified_at
FROM bronze.Address a;

In [ ]:
%%sql
CREATE MATERIALIZED LAKE VIEW IF NOT EXISTS silver.mlv_salesheaders AS
SELECT
  h.SalesOrderID,
  CAST(h.OrderDate AS DATE)          AS OrderDate,
  h.DueDate,
  h.ShipDate,
  h.Status,
  CASE h.Status
    WHEN 1 THEN 'in process'
    WHEN 2 THEN 'approved'
    WHEN 3 THEN 'backordered'
    WHEN 4 THEN 'rejected'
    WHEN 5 THEN 'shipped'
    WHEN 6 THEN 'cancelled'
    ELSE 'unknown'
  END                                 AS StatusText,
  h.OnlineOrderFlag,
  h.CustomerID,
  h.ShipToAddressID,
  h.BillToAddressID,
  h.SubTotal,
  h.TaxAmt,
  h.Freight,  
  /* Compute TotalDue explicitly */
  (h.SubTotal + h.TaxAmt + h.Freight) AS TotalDue,
  year(h.OrderDate)                   AS OrderYear,
  month(h.OrderDate)                  AS OrderMonth,
  /* watermark */
  coalesce(h.ModifiedDate, to_timestamp('1900-01-01 00:00:00')) AS row_modified_at
FROM bronze.SalesOrderHeader h;

In [ ]:
%%sql
CREATE MATERIALIZED LAKE VIEW IF NOT EXISTS silver.mlv_saleslines AS
SELECT
  d.SalesOrderID,
  d.SalesOrderDetailID,
  h.OrderDate,
  h.DueDate,
  h.ShipDate,
  h.Status,
  h.StatusText,
  h.OnlineOrderFlag,
  h.CustomerID,
  /* propagate address natural keys for Warehouse lookups */
  h.ShipToAddressID,
  h.BillToAddressID,
  d.ProductID,
  d.OrderQty,
  d.UnitPrice,
  d.UnitPriceDiscount,
  CAST(d.UnitPrice * (1.0 - d.UnitPriceDiscount) AS DECIMAL(19,4)) AS UnitPriceNet,
  CAST(d.OrderQty  * d.UnitPrice * (1.0 - d.UnitPriceDiscount) AS DECIMAL(19,4)) AS LineAmount,
  h.SubTotal,
  h.TaxAmt,
  h.Freight,
  h.TotalDue,
  h.OrderYear,
  h.OrderMonth,
  /* unified watermark across header + detail */
  COALESCE(
    greatest(h.row_modified_at, d.ModifiedDate),
    h.row_modified_at,
    d.ModifiedDate,
    to_timestamp('1900-01-01 00:00:00')
  ) AS row_modified_at
FROM silver.mlv_salesheaders h
JOIN bronze.SalesOrderDetail d
  ON d.SalesOrderID = h.SalesOrderID;